In [4]:
!wget https://github.com/devic1/RNN-/raw/main/data.zip

--2023-02-11 08:56:20--  https://github.com/devic1/RNN-/raw/main/data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/devic1/RNN-/main/data.zip [following]
--2023-02-11 08:56:20--  https://raw.githubusercontent.com/devic1/RNN-/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23197 (23K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  22.65K  --.-KB/s    in 0s      

2023-02-11 08:56:21 (102 MB/s) - ‘data.zip’ saved [23197/23197]



In [5]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/boys.txt           
  inflating: data/girls.txt          


In [6]:
import torch 
import torch.nn as nn
import os
import unicodedata

In [7]:
device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
device

device(type='cuda')

In [15]:
lett = set()
g = " .,;'"
for i in g:
  lett.add(i)
lett

{' ', "'", ',', '.', ';'}

In [16]:
cat = {}
for i in os.listdir("data"):
  with open(os.path.join("data",i),"r") as f:
    nd = f.read().split("\n")
    for j in nd:
      for k in j:
        lett.add(k)
    cat[i[:-4]] = nd
n_cat = len(cat)

In [17]:
ltoi = {j:i for i,j in enumerate(sorted(lett))}
ln_l = len(ltoi)
print(ltoi)
ln_l

{'\t': 0, ' ': 1, "'": 2, ',': 3, '.': 4, '0': 5, ';': 6, 'ஃ': 7, 'அ': 8, 'ஆ': 9, 'இ': 10, 'உ': 11, 'ஊ': 12, 'எ': 13, 'ஏ': 14, 'ஐ': 15, 'ஒ': 16, 'ஓ': 17, 'க': 18, 'ங': 19, 'ச': 20, 'ஜ': 21, 'ஞ': 22, 'ட': 23, 'ண': 24, 'த': 25, 'ந': 26, 'ன': 27, 'ப': 28, 'ம': 29, 'ய': 30, 'ர': 31, 'ற': 32, 'ல': 33, 'ள': 34, 'ழ': 35, 'வ': 36, 'ஷ': 37, 'ஸ': 38, 'ஹ': 39, 'ா': 40, 'ி': 41, 'ீ': 42, 'ு': 43, 'ூ': 44, 'ெ': 45, 'ே': 46, 'ை': 47, 'ொ': 48, 'ோ': 49, 'ௌ': 50, '்': 51}


52

In [18]:
print(cat['boys'][:5])
n_cat

['அகண்டலன்', 'அகத்தியன்', 'அகரன்', 'அகரமுதல்வன்', 'அகற்கண்ணன்']


2

In [28]:
def enc(letter):
  tempt = torch.tensor([])
  for idx,ele in enumerate(letter):
    c = torch.nn.functional.one_hot(torch.tensor(ltoi[ele]),num_classes=ln_l)
    tempt = torch.cat((tempt,c.unsqueeze(0)),dim=0)
  return tempt.view(-1,1,ln_l)

In [29]:
f = enc("அகண்டலன்")
f[0].shape

torch.Size([1, 52])

In [59]:
catn = list(cat.keys())
def getdata(train=True):
  ranc = torch.randint(0,2,(1,))
  catname = catn[ranc]
  ln = len(cat[catname])
  split = int(0.9*ln)
  if train:
    ranw = torch.randint(0,split,(1,))
  else:
    ranw = torch.randint(split,ln,(1,))
  wtt = cat[catname][ranw]
  if len(wtt) == 0:
    return getdata()
  return ranc,enc(wtt),catname,wtt

In [60]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128

In [61]:
rnn = RNN(ln_l, n_hidden, n_cat).to(device)

In [62]:
loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(rnn.parameters(),lr=0.005)

In [63]:
epochs = 100000
rnn.train()
for i in range(epochs):
  label,inp,name,word = getdata()
  hidden = rnn.initHidden()
  inp,hidden,label = inp.to(device),hidden.to(device),label.to(device)
  for j in range(len(word)):
    output,hidden = rnn(inp[j],hidden)
  loss = loss_fn(output,label)
  optimizer.zero_grad()  
  loss.backward()
  optimizer.step()
  if i % (epochs/10) == 0:
    print(loss)

tensor(0.7171, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3772, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0529, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2324, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0301, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3088, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2646, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1219, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0314, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0220, device='cuda:0', grad_fn=<NllLossBackward0>)


In [64]:
rnn.eval()
epoch = 1000
num_miss = 0
for j in range(epoch):
  label,inp,name,word = getdata(train=False)
  hidden = rnn.initHidden()
  inp,hidden,label = inp.to(device),hidden.to(device),label.to(device)
  for k in range(len(word)):
    output,hidden = rnn(inp[k],hidden)
  h,res = output.topk(1)
  if res != label:
    num_miss += 1

In [65]:
accuracy = 100-((num_miss/epoch)*100)
print(f'The accuracy for the model is {accuracy}%')

The accuracy for the model is 81.7%


In [72]:
n = "விக்டர்"
#n = "த்ரிஷா"
#n = input("Enter the name : ")
inp = enc(n).to(device)
for i in range(len(n)):
  hid = rnn.initHidden().to(device)
  ans,hid = rnn(inp[i],hid)
_,result = ans.topk(1)
print(f'The Name {n} is a {catn[result][:-1]} name.')

The Name விக்டர் is a boy name.


In [73]:
print("Happy Ending :)")

Happy Ending :)
